In [53]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import numpy
import pandas as pd
import os
import pickle 
import io 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score


# Student ID: XXX

**You student_id is your 7/8 digit faser number.** 

This is a sample format for CE807: Assignment 2. You must follow the format.
The code will have three broad sections, and additional section, if needed,


1.   Common Codes
2.   Method/model 1 Specific Codes
3.   Method/model 2 Specific Codes
4.   Other Method/model Codes, if any

**You must have `train_method1`, `test_method1` for method 1 and `train_method2`, `test_method2` for method 2 to perform full training and testing. This will be evaluated automatically, without this your code will fail and no marked.** 

You code should be proverly indended, print as much as possible, follow standard coding (https://peps.python.org/pep-0008/) and documentaion (https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.01-Help-And-Documentation.ipynb) practices. 

Before each `code cell`, you must have a `text cell` which explain what code cell is going to do. For each function/class, you need to properly document what are it's input, functionality and output. 

If you are using any non-standard library, you must have command to install that, for example `pip install datasets`. 

You must print `train`, `validation` and `test` performance measures.

You must also print `train` and `validation` loss in each `epoch`, wherever you are using `epoch`, say in any deep learning algorithms.

Your code must

*   To reproducibality of the results you must use a `seed`, you have to set seed in `torch`, `numpy` etc, use same seed everywhere **and your Student ID should be your seed**. 
*   read dataset from './CE807/Assignment2/student_id/' folder which will have 3 files [`train.csv`, `val.csv`, `test.csv`]
*   save model after finishing the training in './CE807/Assignment2/student_id/models/XXX/' where XXX = [1,2] for both models
*   at testing time you will load models from './CE807/Assignment2/student_id/models/XXX/' where XXX = [1,2] and then test on your data, and save the output in the same folder
*   For Data Size Effect, you model and output save directories are './CE807/Assignment2/student_id/models/XXX/YYY/' where XXX = [1,2] and YYY = [25,50, 75,100]
*   **Your output file based on the test file will be named `output_test.csv` and will have fields `id`, `tweet`, `label` and `out_label`** Note that, `id`, `tweet`, `label` come from `test.csv`, and `out_label` out_label your model’s output, where out_label =[OFF,NOT]. You need to save file in the respective model folders. 




**Install and import all required libraries first before starting to code.**








Let's install all require libraries. For example, `transformers`

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.1 MB/s eta 0:00:00


**Let's put your student id as a variable, that you will use different places**

In [5]:
student_id = 2202396 # Note this is an interger and you need to input your id

Let's set `seed` for all libraries like `torch`, `numpy` etc as my student id

In [6]:
# set same seeds for all libraries

#numpy seed
np.random.seed(student_id)

# Common Codes 

In this section you will write all common codes, for examples


*   Data read
*   Data Splitting
*   Performance Matrics
*   Print Dataset Statistics
*   Saving model and output
*   Loading Model and output
*   etc




**Let's first allow the GDrive access and set data and model paths**

For examples, 

student_id = 1234567

set GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = ‘./CE807/Assignment2/student_id/’ in your GDrive

now set all global variable, 
(Example for model 1 with 25% of data)

train_file = os.path.join(GOOGLE_DRIVE_PATH_AFTER_MYDRIVE, 'train.csv')

MODEL_1_DIRECTORY = os.path.join('gdrive', 'MyDrive', GOOGLE_DRIVE_PATH, ‘models’, ‘1’)

MODEL_1_25_DIRECTORY = os.path.join('MODEL_1_DIRECTORY',‘25’)

model_1_25_output_test_file = os.path.join(MODEL_1_25_DIRECTORY,'output_test.csv')

Sample output directory and file structure: https://drive.google.com/drive/folders/1okgSzgGiwPYYFp7NScEt9MNVolOlld1d?usp=share_link   

In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [77]:
MODEL_1_DIRECTORY = os.path.join(GOOGLE_DRIVE_PATH, 'models', '1') # Model 1 directory
print('Model 1 directory: ', MODEL_1_DIRECTORY)

MODEL_1_25_DIRECTORY = os.path.join(MODEL_1_DIRECTORY,'25') # Model 1 trained using 25% of train data directory
print('Model 1 directory with 25% data: ', MODEL_1_25_DIRECTORY)

MODEL_1_100_DIRECTORY = os.path.join(MODEL_1_DIRECTORY,'100') # Model 1 trained using 25% of train data directory
print('Model 1 directory with 100% data: ', MODEL_1_100_DIRECTORY)


model_1_25_output_test_file = os.path.join(MODEL_1_25_DIRECTORY, 'output_test.csv') # Output file using Model 1 trained using 25% of train data 
print('Output file name using model 1 using 25% of train data: ',model_1_25_output_test_file)

model_1_100_output_test_file = os.path.join(MODEL_1_100_DIRECTORY, 'output_test.csv') # Output file using Model 1 trained using 25% of train data 
print('Output file name using model 1 using 100% of train data: ',model_1_100_output_test_file)

Model 1 directory:  gdrive/MyDrive/./CE807/Assignment2/2202396/models/1
Model 1 directory with 25% data:  gdrive/MyDrive/./CE807/Assignment2/2202396/models/1/25
Model 1 directory with 100% data:  gdrive/MyDrive/./CE807/Assignment2/2202396/models/1/100
Output file name using model 1 using 25% of train data:  gdrive/MyDrive/./CE807/Assignment2/2202396/models/1/25/output_test.csv
Output file name using model 1 using 100% of train data:  gdrive/MyDrive/./CE807/Assignment2/2202396/models/1/100/output_test.csv


Let's see train file

**Cleaning Data set**

In [20]:
# Load data into a pandas DataFrame
df = pd.read_csv('/content/gdrive/MyDrive/CE807/Assignment2/2202396/train.csv')

# Removing duplicates
df = df.drop_duplicates()

# Checking for missing values
print(df.isnull().sum())

# Drop rows with missing values
df = df.dropna()

df.to_csv('/content/gdrive/MyDrive/CE807/Assignment2/2202396/cleaned_data.csv', index=False)

id       0
tweet    0
label    0
dtype: int64


We are going to use different performance matrics like Accuracy, Recall (macro), Precision (macro), F1 (macro) and Confusion Matrix for the performance evaluation. We will print all the matrics and display Confusion Matrix with proper X & Y axis labels

In [ ]:
def compute_performance(y_true, y_pred):
    """
    prints different performance matrics like  Accuracy, Recall (macro), Precision (macro), and F1 (macro).
    This also display Confusion Matrix with proper X & Y axis labels.
    Also, returns F1 score

    Args:
        y_true: numpy array or list
        y_pred: numpy array or list
        

    Returns:
        float
    """

    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    # Replace "pass" statement with your code
    # Evaluate the model performance using metrics such as accuracy, precision, recall, and F1-score
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='offensive')
    recall = recall_score(y_test, y_pred, pos_label='offensive')
    f1 = f1_score(y_test, y_pred, pos_label='offensive')
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################
    return f1_score

In [65]:
def compute_performance(y_true, y_pred, split='test'):
    """
    prints different performance matrics like  Accuracy, Recall (macro), Precision (macro), and F1 (macro).
    This also display Confusion Matrix with proper X & Y axis labels.
    Also, returns F1 score

    Args:
        y_true: numpy array or list
        y_pred: numpy array or list
        split: str
        

    Returns:
        float
    """

    print('Computing different preformance metrics on', split, ' set of Dataset')
    f1score=f1_score(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)
    
    print('F1 Score(macro): ', f1score)
    print('Accuracy: ', acc)

    return f1score

# Method 1 Start

In this section you will write all details of your Method 1. 

You will have to enter multiple `code` and `text` cell.

Your code should follow the standard ML pipeline


*   Data reading
*   Data clearning, if any
*   Convert data to vector/tokenization/vectorization
*   Model Declaration/Initialization/building
*   Training and validation of the model using training and validation dataset 
*   Save the trained model
*   Load and Test the model on testing set
*   Save the output of the model


You could add any other step(s) based on your method's requirement. 

After finishing the above, you need to usd splited data as defined in the assignment and then do the same for all 4 sets. Your code should not be copy-pasted 4 time, make use of `function`.


In [67]:
def prepare_dataset1(data, count_vectorizer=None, split='test'):
  if split == 'train':
      count_vectorizer = CountVectorizer(stop_words='english',max_features=5000) 
      values = count_vectorizer.fit_transform(data['tweet'].values) #TODO: This is the best way to do this, because you need to use same vectorization menthod
  else:
      values = count_vectorizer.transform(data['tweet'].values)

  if split == 'train':
      return values, count_vectorizer
  else:
      return values

In [80]:
def train_model1(text_vector,label):

    print('Let\'s start training RandomForestClassifier')
    classifier = RandomForestClassifier() 
    classifier.fit(text_vector, label)

    return classifier

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [68]:
def save_model1(model, vectorizer, model_dir):
    # save the model to disk
    model_file = os.path.join(model_dir, 'model.sav')
    pickle.dump(model, open(model_file, 'wb'))

    print('Saved model to ', model_file)

    vectorizer_file = os.path.join(model_dir, 'vectorizer.sav') 
    pickle.dump(vectorizer, open(vectorizer_file, 'wb'))

    print('Saved Vectorizer to ', vectorizer_file)

    return model_file, vectorizer_file 

In [69]:
def load_model1(model_file, vectorizer_file):
    # load model and vectorizer from disk

    model = pickle.load(open(model_file, 'rb'))

    print('Loaded model from ', model_file)

    vectorizer = pickle.load(open(vectorizer_file, 'rb'))

    print('Loaded Vectorizer from ', vectorizer_file)


    return model, vectorizer

## Training Method 1 Code
Your test code should be a stand alone code that must take `train_file`, `val_file`,  and `model_dir` as input. You could have other things as also input, but these three are must. You would load both files, and train using the `train_file` and validating using the `val_file`. You will `print` / `display`/ `plot` all performance metrics, loss(if available) and save the output model in the `model_dir`.

Note that at the testing time, you need to use the same pre-processing and model. So, it would be good that you make those as seperate function/pipeline whichever it the best suited for your method. Don't copy-paste same code twice, make it a fucntion/class whichever is best. 

In [54]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import joblib
from sklearn.ensemble import RandomForestClassifier

In [74]:
def train_method1(train_file, val_file, model_dir):
    """
     Takes train_file, val_file and model_dir as input.
     It trained on the train_file datapoints, and validate on the val_file datapoints.
     While training and validating, it print different evaluataion metrics and losses, wheverever necessary.
     After finishing the training, it saved the best model in the model_dir.

     ADD Other arguments, if needed.

    Args:
        train_file: Train file name
        val_file: Validation file name
        model_dir: Model output Directory
    
    """
    train_df = pd.read_csv(train_file)
    val_df = pd.read_csv(val_file)

    train_label = train_df['label']
    val_label = val_df['label']

    train_values, count_vectorizer = prepare_dataset1(train_df, split='train') 
    val_values= prepare_dataset1(val_df,count_vectorizer)

    model = train_model1(train_values,train_label)

    model_file, vectorizer_file = save_model1(model, count_vectorizer, model_dir)

    train_pred_label = model.predict(train_values)
    val_pred_label = model.predict(val_values)

    # print('Train Split')
    train_f1_score = compute_performance(train_label, train_pred_label, split='train')

    # print('Validation Split')
    val_f1_score = compute_performance(val_label, val_pred_label, split='valid')


    return model_file, vectorizer_file

In [72]:
train_file = os.path.join(GOOGLE_DRIVE_PATH, 'train.csv') # This is 100% of data
train_25_file = os.path.join(GOOGLE_DRIVE_PATH, 'train_25.csv') #Let's assume that you have train 25% file is saved in train_25.csv. Note that this is a dummy file. You have to create your own file.


print('Train 100% file: ', train_file)
print('Train 25% file: ', train_25_file)

val_file = os.path.join(GOOGLE_DRIVE_PATH, 'valid.csv')
print('Validation file: ', val_file)

test_file = os.path.join(GOOGLE_DRIVE_PATH, 'test.csv')
print('Test file: ', test_file)

Train 100% file:  gdrive/MyDrive/./CE807/Assignment2/2202396/train.csv
Train 25% file:  gdrive/MyDrive/./CE807/Assignment2/2202396/train_25.csv
Validation file:  gdrive/MyDrive/./CE807/Assignment2/2202396/valid.csv
Test file:  gdrive/MyDrive/./CE807/Assignment2/2202396/test.csv


In [79]:
print('Train using of 100% of data')
model_100_file, vectorizer_100_file = train_method1(train_file, val_file, MODEL_1_100_DIRECTORY)

Train using of 100% of data
Let's start training MultinomialNB
Saved model to  gdrive/MyDrive/./CE807/Assignment2/2202396/models/1/100/model.sav
Saved Vectorizer to  gdrive/MyDrive/./CE807/Assignment2/2202396/models/1/100/vectorizer.sav
Computing different preformance metrics on train  set of Dataset
F1 Score(macro):  0.9933904329025784
Accuracy:  0.9941525217250061
Computing different preformance metrics on valid  set of Dataset
F1 Score(macro):  0.7216294342164347
Accuracy:  0.7691477885652643


In [63]:
train_method1(train_df, valid_df, model_dir)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [57]:
import pickle
from sklearn.linear_model import LogisticRegression

# Define a function to train and save the model
def train_and_save_model(x_train, y_train, model_path):
    # Define the model
    model = LogisticRegression()

    # Train the model
    model.fit(x_train, y_train)

    # Save the model
    with open(model_path, 'wb') as file:
        pickle.dump(model, file)

# Define a function to load the saved model and make predictions
def load_and_predict(X_test, model_path):
    # Load the model
    with open(model_path, 'rb') as file:
        model = pickle.load(file)

    # Make predictions
    y_pred = model.predict(X_test)

    return y_pred

In [58]:
train_and_save_model(train_df, valid_df, model_dir)

ValueError: ignored

## Testing Method 1 Code
Your test code should be a stand alone code that must take `test_file`, `model_file` and `output_dir` as input. You could have other things as also input, but these three are must. You would load both files, and generate output based on inputs. Then you will `print` / `display`/ `plot` all performance metrics, and save the output file in the `output_dir`  

In [ ]:
# Use the trained model to predict the label for new, unseen text data
new_text = "This is an offensive sentence."
new_tokens = word_tokenize(new_text)
new_vec = vectorizer.transform([new_tokens])
new_pred = clf.predict(new_vec)
print("Predicted label:", new_pred[0])

In [ ]:
def test_method1(test_file, model_file, output_dir):
    """
     take test_file, model_file and output_dir as input.
     It loads model and test of the examples in the test_file.
     It prints different evaluation metrics, and saves the output in output directory

     ADD Other arguments, if needed

    Args:
        test_file: test file name
        model_file: model file name
        output_dir: Output Directory
        

    
    """

    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    # Replace "pass" statement with your code
    # Convert tokenized text data into numerical vectors
    vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    test_tokens = [word_tokenize(text) for text in test_df]
    test_vec = vectorizer.transform(test_tokens)

    # Predict the labels for the testing set
    
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################
    return 

## Method 1 End


# Method 2 Start

In this section you will write all details of your Method 2.
 
You will have to enter multiple `code` and `text` cell.

Your code should follow the standard ML pipeline


*   Data reading
*   Data clearning, if any
*   Convert data to vector/tokenization/vectorization
*   Model Declaration/Initialization/building
*   Training and validation of the model using training and validation dataset 
*   Save the trained model
*   Load and Test the model on testing set
*   Save the output of the model

You could add any other step(s) based on your method's requirement. 

After finishing the above, you need to usd splited data as defined in the assignment and then do the same for all 4 sets. Your code should not be copy-pasted 4 time, make use of `function`.


## Training Method 2 Code
Your test code should be a stand alone code that must take `train_file`, `val_file`,  and `model_dir` as input. You could have other things as also input, but these three are must. You would load both files, and train using the `train_file` and validating using the `val_file`. You will `print` / `display`/ `plot` all performance metrics, loss(if available) and save the output model in the `model_dir`.

Note that at the testing time, you need to use the same pre-processing and model. So, it would be good that you make those as seperate function/pipeline whichever it the best suited for your method. Don't copy-paste same code twice, make it a fucntion/class whichever is best. 

In [ ]:
def train_method2(train_file, val_file, model_dir):
  """
     Takes train_file, val_file and model_dir as input.
     It trained on the train_file datapoints, and validate on the val_file datapoints.
     While training and validating, it print different evaluataion metrics and losses, wheverever necessary.
     After finishing the training, it saved the best model in the model_dir.

     ADD Other arguments, if needed.

    Args:
        train_file: Train file name
        val_file: Validation file name
        model_dir: Model output Directory
        

    
    """

    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    # Replace "pass" statement with your code
    pass
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################
    return 

## Testing Method 2 Code
Your test code should be a stand alone code that must take `test_file`, `model_file` and `output_dir` as input. You could have other things as also input, but these three are must. You would load both files, and generate output based on inputs. Then you will `print` / `display`/ `plot` all performance metrics, and save the output file in the `output_dir`  

In [ ]:
def test_method2(test_file, model_file, output_dir):
  """
     take test_file, model_file and output_dir as input.
     It loads model and test of the examples in the test_file.
     It prints different evaluation metrics, and saves the output in output directory

     ADD Other arguments, if needed

    Args:
        test_file: test file name
        model_file: model file name
        output_dir: Output Directory
        

    
    """

    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    # Replace "pass" statement with your code
    pass
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################
    return 

## Method 2 End


# Other Method/model Start

In [ ]:
# your code

##Other Method/model End